In [8]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# [原文地址](https://www.toutiao.com/i6694196616334672387/)

In [1]:
import requests
from bs4 import BeautifulSoup

user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
referer = "http://www.xicidaili.com/"
headers = {"user-agent": user_agent, "referer": referer}
target = "http://www.xicidaili.com/nt/"
# 获取页面源码
r = requests.get(target, headers=headers)

In [2]:
# 解析页面源码
soup = BeautifulSoup(r.text, "lxml")
for tr in soup.find_all("tr")[1:]:
    tds = tr.find_all("td")
    proxy = tds[1].text + ":" + tds[2].text
    print(proxy)

124.207.82.166:8008
121.69.37.6:9797
49.51.155.45:8081
219.131.241.89:9797
59.44.247.194:9797
14.20.235.199:9797
175.102.3.98:8089
113.247.252.116:3128
221.7.211.246:60233
211.101.154.105:43598
125.62.27.53:3128
125.46.0.62:53281
124.205.143.213:32612
113.54.153.217:1080
218.64.69.79:8080
121.17.174.121:9797
114.249.118.244:9000
59.37.18.243:3128
218.66.253.146:8800
114.139.32.61:8118
211.162.70.229:3128
119.123.179.163:9000
113.116.121.77:9000
112.95.25.130:8118
101.251.216.103:8080
175.6.5.234:8888
113.247.252.114:9090
58.240.220.86:53281
163.125.69.146:8888
113.54.157.34:1080
182.111.129.37:53281
14.20.235.92:808
183.63.114.204:54321
121.79.131.58:8080
221.226.94.218:110
58.243.50.184:53281
183.30.204.91:9999
183.129.207.80:21776
60.211.218.78:53281
116.252.39.176:53281
203.110.164.139:52144
124.232.133.199:3128
180.169.186.155:1080
210.26.49.88:3128
110.172.221.241:8080
111.160.169.54:42626
58.249.55.222:9797
221.4.150.7:8181
112.80.41.78:8888
101.231.234.38:8080
182.138.242.113:81

In [3]:
import redis

HOST = "127.0.0.1"  # redis所在主机IP
PORT = 6379  # redis服务监听的端口
PASSWORD = ""  # 连接redis的密码
DB = 0  # IP存储的DB编号


def get_connection():
    r = redis.Redis(host=HOST, port=PORT, password=PASSWORD, db=DB)
    return r

In [9]:
import requests
from bs4 import BeautifulSoup

# 获取redis连接
try:
    conn = get_connection()
except Exception:
    print("Error while connecting to redis")

user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
referer = "http://www.xicidaili.com/"
headers = {"user-agent": user_agent, "referer": referer}
target = "http://www.xicidaili.com/nt/"
# 获取页面源码
r = requests.get(target, headers=headers)
# 解析页面源码
soup = BeautifulSoup(r.text, "lxml")
for tr in soup.find_all("tr")[1:]:
    tds = tr.find_all("td")
    proxy = tds[1].text + ":" + tds[2].text
    conn.sadd("ip_set", proxy)
    print("%s added to ip set" % proxy)

121.17.174.121:9797 added to ip set
123.249.28.188:3128 added to ip set
121.232.199.62:9000 added to ip set
59.37.18.243:3128 added to ip set
111.202.37.195:8080 added to ip set
221.7.255.168:8080 added to ip set
116.196.90.176:3128 added to ip set
221.6.32.214:50514 added to ip set
115.239.248.181:3128 added to ip set
27.191.234.69:9999 added to ip set
14.118.130.214:8081 added to ip set
125.46.0.62:53281 added to ip set
101.231.234.38:8080 added to ip set
121.69.46.177:9000 added to ip set
124.207.82.166:8008 added to ip set
183.129.207.89:27794 added to ip set
1.203.72.115:9000 added to ip set
202.104.150.130:33163 added to ip set
180.175.22.135:9797 added to ip set
112.65.52.28:9000 added to ip set
121.232.199.51:9000 added to ip set
221.7.211.246:60233 added to ip set
14.20.235.201:9797 added to ip set
122.141.74.186:8080 added to ip set
182.111.129.37:53281 added to ip set
61.145.182.27:53281 added to ip set
182.18.13.149:53281 added to ip set
27.46.21.148:8888 added to ip set
12

<IPython.core.display.Javascript object>

In [14]:
# 判断IP是否能访问目标网站
def is_valid(url, ip):
    proxy = {"http": "http://%s" % ip}
    user_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"
    headers = {"user-agent": user_agent}
    try:
        print(proxy)
        print(url)
        r = requests.get(url, headers=headers, proxies=proxy, timeout=6)
        print(r)
        return True
    except Exception:
        return False


my_proxy, proxies, ip_set = None, None, "amazon_ips"
conn = get_connection()
target = "https://www.amazon.com/"

while not is_valid(target, my_proxy):
    if my_proxy:
        conn.srem(ip_set, my_proxy)  # 删除无效的代理IP
    if proxies:
        my_proxy = proxies.pop()
    else:
        proxies = conn.srandmember(ip_set, 5)  # 从redis中随机抽5个代理ip
        my_proxy = proxies.pop()
        print("valid proxy %s" % my_proxy)

{'http': 'http://None'}
https://www.amazon.com/
<Response [200]>


<IPython.core.display.Javascript object>